In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Mon Apr 28 16:33:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.58.02              Driver Version: 555.58.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   24C    P0             60W /  500W |     138MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 84.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 KB 124.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 KB 117.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 156.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 114.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 136.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.29.3
    Uninstalling huggingface-hub-0.29.3:
      Successfully uninstalled huggingface-hub-0.29.3


In [4]:
!pip install torch tensorboard protobuf peft sentencepiece datasets bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 127.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.2/316.2 KB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 KB 129.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 163.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 KB 126.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 34.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 101.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 15.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 19.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [5]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [6]:
TOKEN = "" # HF token
MODEL_NAME = "laco-layerskip-llama2-7b"
MODEL_SAVE_DIR = "./laco-layerskip-llama2-7b-finetuned"
DEVICE = "cuda:0"
SEED = 123

In [7]:
import torch
import math

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

from datasets import load_dataset
from peft import LoraConfig, get_peft_model

from huggingface_hub import login, snapshot_download

login(token=TOKEN)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True,
    token=TOKEN
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, 
    token=TOKEN,
    use_fast=True
)

model = model.to(DEVICE)

In [6]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

### Fine-tuning

In [ ]:
wikipedia_dataset = load_dataset("wikimedia/wikipedia", "20231101.en", split="train")
train_size = 7000
val_size = 1500
wikipedia_dataset = wikipedia_dataset.shuffle(seed=SEED)
wikipedia_dataset_subset = wikipedia_dataset.select(range(train_size))

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=2048)

tokenized_wiki = wikipedia_dataset_subset.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=["text"],
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [ ]:
val_dataset = wikipedia_dataset.select(range(train_size, train_size + val_size))
tokenized_val = val_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=["text"],
)

val_dataset_ready = tokenized_val

In [15]:
total_train_tokens = sum(len(example["input_ids"]) for example in tokenized_wiki)
print(f"Total train tokens: {total_train_tokens}")

total_val_tokens = sum(len(example["input_ids"]) for example in val_dataset_ready)
print(f"Total val tokens: {total_val_tokens}")

Total train tokens: 4472103
Total val tokens: 976274


In [16]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

In [17]:
training_args = TrainingArguments(
    output_dir=MODEL_SAVE_DIR,
    per_device_train_batch_size=2, 
    gradient_accumulation_steps=2,  
    max_steps=3600,
    logging_steps=50,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=250,
    save_total_limit=1,
    bf16=True,  
    optim="paged_adamw_32bit", 
    report_to="tensorboard",
    logging_dir="/workspace/tensorboard_laco_layerskip_llama",
    group_by_length=True,
    learning_rate=2e-4,
    warmup_steps=100,
    weight_decay=0.0,
    lr_scheduler_type="cosine",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wiki,
    eval_dataset=val_dataset_ready,
    data_collator=data_collator,
)

trainer.train()

# training_args = TrainingArguments(
#     output_dir="./llama2-7b-finetuned",
#     per_device_train_batch_size=2, 
#     gradient_accumulation_steps=8,   
#     num_train_epochs=2,
#     logging_steps=10,
#     save_steps=100,
#     save_total_limit=2,
#     eval_strategy="no",
#     bf16=True,
#     optim="adamw_torch",
#     learning_rate=2e-4,
#     weight_decay=0.01,
#     max_grad_norm=1.0,
#     lr_scheduler_type="cosine",
#     warmup_ratio=0.05,
#     ddp_find_unused_parameters=False,
#     report_to="none",
# )

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
250,1.969800,1.935256
500,1.902900,1.905965
750,1.862400,1.890535
1000,1.872900,1.875160
1250,1.840400,1.865805
1500,1.862100,1.856319
1750,1.889100,1.850141
2000,1.835100,1.849035
2250,1.805200,1.845148
2500,1.829700,1.840263


TrainOutput(global_step=3600, training_loss=1.8541072252061632, metrics={'train_runtime': 1835.413, 'train_samples_per_second': 7.846, 'train_steps_per_second': 1.961, 'total_flos': 2.1034469407408128e+17, 'train_loss': 1.8541072252061632, 'epoch': 2.057142857142857})

In [18]:
trainer.save_model(MODEL_SAVE_DIR)
tokenizer.save_pretrained(MODEL_SAVE_DIR)

('./laco-layerskip-llama2-7b-finetuned/tokenizer_config.json',
 './laco-layerskip-llama2-7b-finetuned/special_tokens_map.json',
 './laco-layerskip-llama2-7b-finetuned/tokenizer.model',
 './laco-layerskip-llama2-7b-finetuned/added_tokens.json',
 './laco-layerskip-llama2-7b-finetuned/tokenizer.json')

In [19]:
metrics = trainer.evaluate(eval_dataset=val_dataset_ready)
eval_loss = metrics["eval_loss"]
eval_perplexity = math.exp(eval_loss)

print(f"Validation Loss: {eval_loss:.4f}")
print(f"Validation Perplexity: {eval_perplexity:.2f}")

Validation Loss: 1.8362
Validation Perplexity: 6.27


In [ ]:
encoded_inputs = tokenizer(['Meta vs google, the winner is'], return_tensors='pt').to(DEVICE)

output = model.generate(
    **encoded_inputs,
    max_new_tokens=32,
    do_sample=False,     
    temperature=0     
)

output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(output_text)

In [ ]:
model.push_to_hub("sidsr/laco-llama2-7b-finetuned")
tokenizer.push_to_hub("sidsr/laco-llama2-7b-finetuned")

In [8]:
!pip install lm-eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 29.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 KB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.3/397.3 KB 120.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 KB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 KB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 KB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 KB 101.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 130.4 MB/s eta 0:00:0000:01
doneing metadata (setup.py) ... 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 117.9 MB/s eta 0:00:0000:0100:01
doneing metadata (setup.py) ... 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 141.4 

In [ ]:
snapshot_download("sidsr/laco-layerskip-llama2-7b", local_dir="/workspace/layerskip-modeling/laco-layerskip-llama2-7b")
snapshot_download("sidsr/laco-layerskip-llama2-7b-finetuned", local_dir="/workspace/layerskip-modeling/laco-layerskip-llama2-7b-finetuned")

In [24]:
!ls "/workspace/layerskip-modeling/eval/"

laco-layerskip-llama2-7b	    laco-llama2-7b
laco-layerskip-llama2-7b-finetuned  laco-llama2-7b-finetuned


In [8]:
!mkdir -p "/workspace/layerskip-modeling/eval/laco-llama2-7b"

In [18]:
!ls -al "/workspace/layerskip-modeling/eval/laco-llama2-7b/.__laco-llama2-7b-finetuned"

total 100
drwxrwsr-x+ 2 root user     61 Apr 28 16:53 .
drwxrwsr-x+ 4 root user     78 Apr 28 16:53 ..
-rw-rw-r--+ 1 root user 100578 Apr 28 16:53 results_2025-04-28T16-53-40.913934.json


In [25]:
!lm_eval \
    --model hf \
    --model_args "pretrained=./laco-layerskip-llama2-7b" \
    --tasks mmlu \
    --output_path "/workspace/layerskip-modeling/eval/laco-layerskip-llama2-7b"

2025-04-28:17:52:25,522 INFO     [lm_eval.__main__:379] Selected Tasks: ['mmlu']
2025-04-28:17:52:25,524 INFO     [lm_eval.evaluator:169] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-04-28:17:52:25,524 INFO     [lm_eval.evaluator:206] Initializing hf model, with arguments: {'pretrained': './laco-layerskip-llama2-7b'}
2025-04-28:17:52:25,865 INFO     [lm_eval.models.huggingface:136] Using device 'cuda'
2025-04-28:17:52:25,936 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cuda'}
2025-04-28:17:53:15,174 INFO     [lm_eval.api.task:420] Building contexts for mmlu_abstract_algebra on rank 0...
100%|████████████████████████████████████████| 100/100 [00:00<00:00, 650.55it/s]
2025-04-28:17:53:15,332 INFO     [lm_eval.api.task:420] Building contexts for mmlu_anatomy on rank 0...
100%|████████████████████████████████████████| 

In [22]:
!lm_eval \
    --model hf \
    --model_args "pretrained=./laco-llama2-7b" \
    --tasks mmlu \
    --output_path "/workspace/layerskip-modeling/eval/laco-llama2-7b"

2025-04-28:17:41:52,813 INFO     [lm_eval.__main__:379] Selected Tasks: ['mmlu']
2025-04-28:17:41:52,815 INFO     [lm_eval.evaluator:169] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-04-28:17:41:52,816 INFO     [lm_eval.evaluator:206] Initializing hf model, with arguments: {'pretrained': './laco-llama2-7b'}
2025-04-28:17:41:53,133 INFO     [lm_eval.models.huggingface:136] Using device 'cuda'
2025-04-28:17:41:53,231 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cuda'}
2025-04-28:17:42:47,577 INFO     [lm_eval.api.task:420] Building contexts for mmlu_abstract_algebra on rank 0...
100%|████████████████████████████████████████| 100/100 [00:00<00:00, 653.69it/s]
2025-04-28:17:42:47,735 INFO     [lm_eval.api.task:420] Building contexts for mmlu_anatomy on rank 0...
100%|████████████████████████████████████████| 135/135 [0

In [26]:
!lm_eval \
    --model hf \
    --model_args "pretrained=./laco-layerskip-llama2-7b,peft=./laco-layerskip-llama2-7b-finetuned" \
    --tasks mmlu \
    --output_path "/workspace/layerskip-modeling/eval/laco-layerskip-llama2-7b-finetuned"

2025-04-28:17:58:18,751 INFO     [lm_eval.__main__:379] Selected Tasks: ['mmlu']
2025-04-28:17:58:18,753 INFO     [lm_eval.evaluator:169] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-04-28:17:58:18,753 INFO     [lm_eval.evaluator:206] Initializing hf model, with arguments: {'pretrained': './laco-layerskip-llama2-7b', 'peft': './laco-layerskip-llama2-7b-finetuned'}
2025-04-28:17:58:19,075 INFO     [lm_eval.models.huggingface:136] Using device 'cuda'
2025-04-28:17:58:19,143 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cuda'}
2025-04-28:17:59:09,758 INFO     [lm_eval.api.task:420] Building contexts for mmlu_abstract_algebra on rank 0...
100%|████████████████████████████████████████| 100/100 [00:00<00:00, 660.52it/s]
2025-04-28:17:59:09,915 INFO     [lm_eval.api.task:420] Building contexts for mmlu_anatomy on rank 0...

In [21]:
!lm_eval \
    --model hf \
    --model_args "pretrained=./laco-llama2-7b,peft=./laco-llama2-7b-finetuned" \
    --tasks mmlu \
    --output_path "/workspace/layerskip-modeling/eval/laco-llama2-7b-finetuned"

2025-04-28:17:33:23,628 INFO     [lm_eval.__main__:379] Selected Tasks: ['mmlu']
2025-04-28:17:33:23,630 INFO     [lm_eval.evaluator:169] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-04-28:17:33:23,630 INFO     [lm_eval.evaluator:206] Initializing hf model, with arguments: {'pretrained': './laco-llama2-7b', 'peft': './laco-llama2-7b-finetuned'}
2025-04-28:17:33:23,990 INFO     [lm_eval.models.huggingface:136] Using device 'cuda'
2025-04-28:17:33:24,089 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cuda'}
/venv/main/lib/python3.10/site-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.18.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.18.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1